In [1]:
data = [
['laptop', 'earphones','laptop bag', 'mouse', 'pendrive'],
['mobile', 'cover', 'earphones', 'pendrive' ],
['laptop', 'laptop bag', 'cover', 'pendrive', 'screenguard'],
['earphones', 'mouse', 'pendrive', 'cover', 'screenguard'],
['laptop bag', 'cover', 'pendrive', 'laptop', 'screenguard'],
[ 'laptop','laptop bag','mobile','earphones','pendrive'],
[ 'laptop','laptop bag','mouse','earphones','pendrive'],
[ 'mobile','laptop bag','laptop','laptop bag'],
[ 'laptop','laptop bag','earphones', 'screenguard'],
[ 'laptop','laptop bag','earphones','pendrive'],
['earphones','laptop','laptop bag','cover','mouse'],
['laptop','laptop bag','mobile','pendrive'],
['pendrive','mobile','cover','laptop','mouse'],
['laptop','laptop bag','mouse','pendrive','earphones'],
['earphones','laptop','mobile','laptop bag', 'screenguard'],
]

In [2]:
from numpy import *

In [3]:
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        #print(transaction)
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a key of a dictionary.
    return list(map(frozenset, c1))

In [4]:
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1
    print(sscnt)
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data


In [5]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort() 
            L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
                #print(retList)
        #print(retList)
    return retList

In [6]:
def apriori(dataSet, minSupport = 0.40):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
        
       
    return L, supportData

In [7]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            print(freqSet)
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [8]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [9]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [10]:
L,suppData= apriori(data,minSupport=0.7)

{frozenset({'earphones'}): 10, frozenset({'laptop'}): 13, frozenset({'laptop bag'}): 12, frozenset({'mouse'}): 6, frozenset({'pendrive'}): 11, frozenset({'cover'}): 6, frozenset({'mobile'}): 6, frozenset({'screenguard'}): 5}
{frozenset({'laptop bag', 'pendrive'}): 8, frozenset({'laptop', 'pendrive'}): 9, frozenset({'laptop bag', 'laptop'}): 12}
{}


In [11]:
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'laptop bag', 'laptop'})
frozenset({'laptop'}) --> frozenset({'laptop bag'}) conf: 0.9230769230769231
frozenset({'laptop bag'}) --> frozenset({'laptop'}) conf: 1.0
